In [1]:
import os
import requests
import json
from bs4 import BeautifulSoup as Soup
import pandas as pd

In [2]:
query = "tourism+AND+innovation"
apiKey = "1f3a1599c9ddf0e2afc5dc94b3723bee"

In [3]:
data = {'Title': [],
        'Authors': [],
        'PublicationName': [],
        'Type': [],
        'Abstract': [],
        'Volume': [],
        'Issue': [],
        'Date': [],
        'EID': [],
        'Keywords' : [],
        'URL' : [],
        'OpenAccess': [],
        'References': [],
        'CitedBy': [],
        'AuthorAUID': [],
        'SubjectAreas': []
        }

In [4]:
cursor = "*"
count = 0

while cursor:
    print(count)
    response = requests.get("https://api.elsevier.com/content/search/scopus?query=TITLE-ABS-KEY({})&apiKey={}&cursor={}&count=25"
                            .format(query, apiKey, cursor))
    cont = json.loads(response.content)
    cursor = cont["search-results"]["cursor"]["@next"].replace('+','%2B').replace('/', '%2F')
    for i in cont["search-results"]["entry"]:
        url = i['prism:url']
        data['URL'].append(str(url))
        article = requests.get("{}?apiKey={}".format(url, apiKey))
        soup = Soup(article.content, features="lxml")
        
        data['Title'].append(soup.find('dc:title').get_text().replace('\n', ' ') if soup.find('dc:title') else '')
        authors = soup.find('authors')
        data['Authors'].append('; '.join([item.get_text() for item in authors.find_all('ce:indexed-name') 
                                          if item.parent.name == 'author']) if authors else '')
        data['PublicationName'].append(soup.find('prism:publicationname').get_text() if 
                                       soup.find('prism:publicationname') else '')
        data['Type'].append(soup.find('prism:aggregationtype').get_text() if
                            soup.find('prism:aggregationtype') else '')
        data['Abstract'].append(soup.find('ce:para').get_text().replace('\n', ' ') 
                                if soup.find('ce:para') and soup.find('ce:para').parent.name == 'abstract' else '')
        data['Volume'].append(soup.find('prism:volume').get_text() if soup.find('prism:volume') else '')
        data['Issue'].append(soup.find('prism:issueidentifier').get_text() 
                             if soup.find('prism:issueidentifier') else '')
        data['Date'].append(soup.find('prism:coverdate').get_text() if soup.find('prism:coverdate') else '')
        data['EID'].append(soup.find('eid').get_text() if soup.find('eid') else '')
        data['Keywords'].append('; '.join([label.get_text() for label in soup.find_all('author-keyword') 
                                             if label.parent.name == 'author-keywords']))
        data['OpenAccess'].append(soup.find('openaccess').get_text() if soup.find('openaccess') else '')
        data['References'].append('; '.join([label.get_text() for label in soup.find_all('ref-fulltext')]) 
                                  if soup.find_all('ref-fulltext') else '')
        data['CitedBy'].append(soup.find('citedby-count').get_text() if soup.find('citedby-count') else '')
        data['AuthorAUID'].append('; '.join([auth['auid'] for auth in soup.find_all('author') 
                                                 if auth.parent.name == 'authors']) if soup.find_all('author') else '')
        data['SubjectAreas'].append('; '.join([label.get_text() for label in soup.find_all('subject-area') 
                                                 if label.parent.name == 'subject-areas']) 
                                      if soup.find_all('subject-area') else '')
    count += 25
    df = pd.DataFrame.from_dict(data)
    df.to_csv(os.path.join(os.getcwd(), '../Innovation/Innovation-Scopus.csv'), index=False)

0
25
50


KeyboardInterrupt: 